In [1]:
%reset -f
import os
import sys
import os.path
from osgeo import gdal
from pyproj import Proj, transform
import pandas as pd
import numpy as np
from parflow import Run
from parflow.tools.fs import get_absolute_path
from parflow.tools.io import write_pfb, read_pfb
from matplotlib import pyplot as plt

In [2]:
#edit CLM input file 'drv_clmin.dat'. 
clm_input_dir = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs'   #@Danielle  spinup_WY2003/run_inputs/
drv_clmin_name = 'drv_clmin.dat'  #@Danielle
startcode_num = '1'  #@Danielle
# should be string, 
# if your run is from the beginning of the water year (initial conditions), set it as 2, 
# if your run is from anytime except the beginning of the water year and you are using a restart, set it as 1
clm_ic_num = '1'  #@Danielle
# should be string
# if this is your first run, set it as 2, 
# if this a run restart from a previous one, set it as 1

#don't touch below unless the code can't run
#=============================================
if clm_input_dir[-1] == '/':
        clm_input_dir = clm_input_dir[:-1]
clm_path = clm_input_dir + '/' + drv_clmin_name
print(clm_path)

f1 = open(clm_path,'r')
f2 = open(clm_path + '.bak','w')
for line in f1:
    line = line.strip()
    if line[0:9] == 'startcode':
        new_line = 'startcode      ' + startcode_num + '                                      1=restart file,2=defined\n'
        f2.write(line.replace(line,new_line))
    elif line[0:6] == 'clm_ic':
        new_line = 'clm_ic         ' + clm_ic_num + '                                      1=restart file,2=defined\n'
        f2.write(line.replace(line,new_line))
    else:
        f2.write(line + '\n')
f1.close()
f2.close()
os.remove(clm_path) 
os.rename(clm_path + '.bak',clm_path) 

/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/drv_clmin.dat


In [3]:
# get the restart timestep
# set your path of clm_restart.tcl
data_path = f'{clm_input_dir}/clm_restart.tcl'  #@Danielle #spinup_WY2003/run_inputs/

#don't touch the below unless there is something wrong
#=============================================
data = pd.read_csv(data_path,sep='\s+',header=None)
restart_time = np.array(data)[0,2]
restart_time = restart_time//24 * 24
print(restart_time)

216


In [4]:
# rename restart file
# to the path where the clm.rst.xxxxx.x files located
# It's better to put the restart file in a new folder, otherwise, it may also rename some other files with 00000
# If you restart from the beginning of a water year, no need to do this step
your_path = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/clm-restarts'  #the path @Danielle #spinup_WY2003/run_inputs/clm-restarts

#don't touch the below unless there is something wrong
#=============================================
cd_path = 'cd ' + your_path
os.system(cd_path)
num1 = '00000'
num2 = str(restart_time).zfill(5)
command = 'rename ' + num1 + ' ' + num2 + ' ' + ' *'
print(f'In a terminal window, navigate to the directory where your restarts are located and enter this command: {command}')
#os.system(command)

In a terminal window, navigate to the directory where your restarts are located and enter this command: rename 00000 00216  *


In [12]:
#edit input python script